In [1]:
import os
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
import zipfile
from datetime import datetime
from pandas_gbq import *
from pandas_gbq import to_gbq
from pandas_gbq import read_gbq
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
## define project and set up paths for Google Big Query
service_path = "C:/Users/breni/Documents/"
service_file = 'niekampbreannawedge-8bbebeea1dda.json'
project_id = 'niekampbreannawedge'
data_id = 'wedge24'

beans_key = service_path + service_file

In [3]:

credentials = service_account.Credentials.from_service_account_file(beans_key)

client = bigquery.Client(credentials= credentials, project= project_id)

In [4]:
conn = sqlite3.connect('wedge_summary.db')

# Task 3: Building Summary Tables

In this task, summary files will be created that allow quick answers to certain questions. In order to do this, a single SQLite database will be created in python (in a .db file) containing three tables as follows: 
1. Sales by date by hour
2. Sales by Owner by Year by Month
3. Sales by Product Description by Year by Month

This database will be built with Python below!

In [ ]:
## Query 1: Sales by Date by Hour: By calendar date (YYYY-MM-DD) and hour of the day, 
# determine the total spend in the store, 
# the number of transactions, 
# and a count of the number of items.

query1 = """
    SELECT 
        EXTRACT(DATE FROM datetime) as sale_date, 
        EXTRACT(HOUR FROM datetime) as sale_hour,
        ROUND(SUM(total), 2) as total_sales,
        COUNT(DISTINCT CONCAT(
            CAST(EXTRACT(DATE FROM datetime) AS STRING),
            CAST(register_no AS STRING),
            CAST(emp_no AS STRING),
            CAST(trans_no AS STRING)
        )) AS num_transactions,
        SUM(
            CASE
                WHEN trans_status IN ('V', 'R') THEN -1
                ELSE 1
            END
        ) AS total_items
    FROM `umt-msba.wedge_transactions.transArchive*` AS trans
    JOIN `umt-msba.wedge_transactions.department_lookup` AS depts
        ON trans.department = depts.department
    WHERE trans.department NOT IN (0, 15)
        AND card_no != 3
        AND (trans_status IS NULL
            OR trans_status IN ('V', 'R', '', ' '))
    GROUP BY sale_date, sale_hour
    ORDER BY sale_date, sale_hour;
    
"""

df_transactions = read_gbq(query1, project_id= project_id)

In [ ]:
df_transactions

In [ ]:
# saving the table
df_transactions.to_sql('sales_by_date_hour', conn, if_exists='append', index=False)

In [ ]:
## TASK 2: Sales by Owner by Year by Month: A file that has the following columns: card_no, year, month, sales, transactions, and items.
query2 = """
    SELECT 
        EXTRACT(YEAR FROM datetime) as year, 
        EXTRACT(MONTH FROM datetime) as month,
        ROUND(SUM(total), 2) as total_sales,
        COUNT(DISTINCT CONCAT(
            CAST(EXTRACT(DATE FROM datetime) AS STRING),
            CAST(register_no AS STRING),
            CAST(emp_no AS STRING),
            CAST(trans_no AS STRING)
        )) AS num_transactions,
        SUM(
            CASE
                WHEN trans_status IN ('V', 'R') THEN -1
                ELSE 1
            END
        ) AS total_items
    FROM `umt-msba.wedge_transactions.transArchive*` AS trans
    JOIN `umt-msba.wedge_transactions.department_lookup` AS depts
        ON trans.department = depts.department
    WHERE trans.department NOT IN (0, 15)
        AND card_no != 3
        AND (trans_status IS NULL
            OR trans_status IN ('V', 'R', '', ' '))
    GROUP BY card_no, year, month
    ORDER BY year, month;
    
"""

df_sales_by_owner = read_gbq(query2, project_id= project_id)

In [ ]:
df_sales_by_owner

In [ ]:
# saving 
df_sales_by_owner.to_sql('sales_by_owner_year_month', conn, if_exists='append', index=False)


In [ ]:
## TASK 3: : A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items

query3 = """
    SELECT 
        trans.upc,
        trans.description,
        trans.department as department_num,
        depts.dept_name,
        EXTRACT(YEAR FROM trans.datetime) as year,
        EXTRACT(MONTH FROM trans.datetime) as month,
        ROUND(SUM(trans.total), 2) as total_sales,
        COUNT(DISTINCT CONCAT(
            CAST(EXTRACT(DATE FROM trans.datetime) AS STRING),
            CAST(trans.register_no AS STRING),
            CAST(trans.emp_no AS STRING),
            CAST(trans.trans_no AS STRING)
        )) AS num_transactions,
        SUM(
            CASE
                WHEN trans.trans_status IN ('V', 'R') THEN -1
                ELSE 1
            END
        ) AS total_items
    FROM `umt-msba.wedge_transactions.transArchive*` AS trans
    JOIN `umt-msba.wedge_transactions.department_lookup` AS depts
        ON trans.department = depts.department
    WHERE trans.department NOT IN (0, 15)
        AND (CAST(trans.upc AS STRING) != '0' 
            OR trans.upc IS NULL)
    AND (CAST(trans.card_no AS STRING) != '3' 
        OR trans.card_no IS NULL)
    AND (trans.trans_status IS NULL
        OR trans.trans_status IN ('V', 'R', '', ' '))
GROUP BY trans.upc, trans.description, trans.department, depts.dept_name, year, month
ORDER BY year, month;
    
"""

df_sales_by_product = read_gbq(query3, project_id=project_id)

In [ ]:
df_sales_by_product

In [ ]:
# saving
df_sales_by_product.to_sql('sales_by_product_year_month', conn, if_exists='append', index=False)

In [14]:
### always close the connection!!!
conn.close()

In [ ]:
### Ensuring that our db file was created and is in working fashion
 
conn = sqlite3.connect('wedge_summary.db')

# List all tables in db
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)
print("Tables in the database:")
print(tables)

# Query and display data from the tables
df = pd.read_sql("SELECT * FROM sales_by_date_hour LIMIT 5;", conn)
print("First 5 rows from sales_by_date_hour:")
print(df)

df2 = pd.read_sql("SELECT * FROM sales_by_owner_year_month LIMIT 5;", conn)
print("First 5 rows from sales_by_owner_year_month:")
print(df2)

df3 = pd.read_sql("SELECT * FROM sales_by_product_year_month LIMIT 5;", conn)
print("First 5 rows from sales_by_product_year_month:")
print(df3)

# Close the connection when done
conn.close()

## Now that we have that working -- we can address our attention to the following questions: 

How have our sales-by-day changed over the last few months?

What is our most popular item in each department?

Which owners spend the most per month in each department?

In [ ]:
# Connect to your SQLite database
conn = sqlite3.connect('wedge_summary.db')

# Query 1: Sales by day trend
query_sales_by_day = """
    SELECT sale_date, SUM(total_sales) AS daily_sales
    FROM sales_by_date_hour
    GROUP BY sale_date
    ORDER BY sale_date DESC
    LIMIT 90;
"""
df_sales_by_day = pd.read_sql(query_sales_by_day, conn)
print("Sales by Day (Last 90 Days):")
print(df_sales_by_day)


In [ ]:
# Query 2: Most popular item in each department
query_popular_item = """
    WITH item_sales AS (
        SELECT upc, description, department_num, dept_name, SUM(total_items) AS total_items_sold
        FROM sales_by_product_year_month
        GROUP BY upc, description, department_num, dept_name
    )
    SELECT department_num, dept_name, upc, description, MAX(total_items_sold) AS most_items_sold
    FROM item_sales
    GROUP BY department_num, dept_name;
"""
df_popular_item = pd.read_sql(query_popular_item, conn)
print("\nMost Popular Item in Each Department:")
print(df_popular_item)


In [ ]:
# Query 3: Top spending owners per month and department
query_top_owners = """
    WITH owner_spending AS (
        SELECT card_no, year, month, department_num, dept_name, SUM(total_sales) AS total_spent
        FROM sales_by_product_year_month
        GROUP BY card_no, year, month, department_num, dept_name
    )
    SELECT card_no, year, month, department_num, dept_name, MAX(total_spent) AS most_spent
    FROM owner_spending
    GROUP BY card_no, year, month, department_num, dept_name
    ORDER BY most_spent DESC
    LIMIT 10;
"""
df_top_owners = pd.read_sql(query_top_owners, conn)
print("\nTop Spending Owners per Month and Department:")
print(df_top_owners)

# Close the database connection
conn.close()
